In [7]:
from google.colab import drive

drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [43]:
!pip install bs4 selenium pretty_midi music21 replit-play

     |████████████████████████████████| 972kB 15.6MB/s 
     |████████████████████████████████| 11.8MB 16.9MB/s 


In [9]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [4]:
url = "https://bushgrafts.com/midi/"

In [45]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
from pretty_midi import PrettyMIDI
import glob
from sklearn.preprocessing import OneHotEncoder
from mido import MidiFile
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
from keras.callbacks import ModelCheckpoint 
from keras.utils import np_utils
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Flatten
from tensorflow.keras.layers import MaxPooling2D, BatchNormalization, Activation
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from collections import namedtuple
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [22]:
jazz_midi_files = glob.glob("gdrive/MyDrive/Jazz Midi/data/*mid")
print(jazz_midi_files)

['gdrive/MyDrive/Jazz Midi/data/We Will Meet Again.mid', 'gdrive/MyDrive/Jazz Midi/data/What is there to say.mid', 'gdrive/MyDrive/Jazz Midi/data/What is this thing - solo.mid', 'gdrive/MyDrive/Jazz Midi/data/Where is Love - solo.mid', 'gdrive/MyDrive/Jazz Midi/data/Witchcraft.mid', 'gdrive/MyDrive/Jazz Midi/data/WhoCanI.mid', 'gdrive/MyDrive/Jazz Midi/data/Whilewereyoung.mid', 'gdrive/MyDrive/Jazz Midi/data/whydidi.mid', 'gdrive/MyDrive/Jazz Midi/data/WhyDoILoveYou.mid', 'gdrive/MyDrive/Jazz Midi/data/Yesterday.mid', 'gdrive/MyDrive/Jazz Midi/data/Yesterdays1.mid', 'gdrive/MyDrive/Jazz Midi/data/You Are So Beautiful The Grand.mid', 'gdrive/MyDrive/Jazz Midi/data/youdontknow trio.mid', 'gdrive/MyDrive/Jazz Midi/data/youtookadvantageofme.mid', 'gdrive/MyDrive/Jazz Midi/data/You Stepped Out of a Dream (Doug McKenzie).mid', 'gdrive/MyDrive/Jazz Midi/data/youhavechanged.mid', 'gdrive/MyDrive/Jazz Midi/data/Young and Foolish.mid', 'gdrive/MyDrive/Jazz Midi/data/Zingaro - Jobim.mid']


In [96]:
class AudioGenerator():
  def __init__(self,
               url="https://bushgrafts.com/midi/",
               epochs=50,
               batch_size=32):
    
    self.url = url

    self.epochs = epochs

    self.batch_size = batch_size

    # _ = self.scrape_website()

    jazz_midi_files = glob.glob("gdrive/MyDrive/Jazz Midi/data/*mid")

    the_notes = self.midiToNotes()

    num_vocab = len(set(the_notes))

    pitchs, the_net_input, net_output = self.create_seqeunces(the_notes, num_vocab)


    # model = self.build_model(the_net_input, num_vocab)

    # self.train_model(model, the_net_input, net_output)

    net_input = self.get_inputSequences(the_notes, pitchs, num_vocab)

    audio = self.generate_audio(the_net_input, pitchs, num_vocab)
    self.create_midi(audio)






  # def scrape_website(self, url=None):
  #   if url == None:
  #     url = self.url
    
  #   chrome_options = webdriver.ChromeOptions()
  #   # prefs = {"download.default_director" : r"C:\Users\keval\Coding Projects\Lofi LTSM Generator\data\\"}
  #   prefs = {"download.default_directory" : r"C:\Users\keval\Coding Projects\Lofi LTSM Generator\data\lofi midi",
  #           "directory_upgrade": True}
  #   chrome_options.add_experimental_option("prefs", prefs)
  #   chrome_options.add_argument("--headless")
  #   chrome_options.add_argument("--window-size=%s" % "1920,1080")

  #   driver = webdriver.Chrome(executable_path=r"C:\Users\keval\Coding Projects\Lofi LTSM Generator\utilities\chromedriver", chrome_options=chrome_options) # chrome_options=chrome_options
  #   driver.get(url)
  #   download_links = driver.find_elements_by_tag_name("a")

  #   hrefs = []
  #   for link in download_links:
  #       href = link.get_attribute("href")
  #       hrefs.append(href)

  #   for href in hrefs:
  #       if href is not None:
  #           driver.get(href)
  #       else:
  #           print("href is none value")
    
  #   return hrefs

  def midiToNotes(self):
    the_notes = []
    for jazz_file in jazz_midi_files:
        jazz_midi = converter.parse(jazz_file)

        jazz_midi_notes_to_parse = []

        instrument_parts = None
        
        try:
            instrument_parts = instrument.partitionByInstrument(midi)  
        except:
            pass

        if instrument_parts == None:
            parse_notes = jazz_midi.flat.notes
        else:
            parse_notes = instrument_parts.parts[0].recurse()

        for the_note in parse_notes:  
            if isinstance(the_note, note.Note):   
                the_notes.append(str(the_note.pitch))

            elif(isinstance(the_note, chord.Chord)):  
                the_notes.append('.'.join(str(n) for n in the_note.normalOrder))  

        with open('notes', 'wb') as filepath:  
            pickle.dump(the_notes, filepath)
      
    return the_notes
  
  def create_seqeunces(self, the_notes, length_of_seqeunce):
    pitchs = sorted(set(the_note for the_note in the_notes))
    note_int = dict((the_note, number) for number, the_note in enumerate(pitchs))

    net_input = []
    net_output = []
    for i in range(0, len(the_notes) - length_of_seqeunce, 1):
      inputs = the_notes[i: i+length_of_seqeunce]
      output = the_notes[i+length_of_seqeunce]
      net_input.append([note_int[input] for input in inputs])
      net_output.append(note_int[output])
    
    num_of_patterns = len(net_input)
    net_input = np.reshape(net_input, (num_of_patterns, length_of_seqeunce, 1))

    n_vocab = len(set(the_notes))

    net_input - net_input / float(n_vocab)

    net_output = np_utils.to_categorical(net_output)

    return pitchs, net_input, net_output

  def build_model(self, net_input, num_vocab):
     model = Sequential()  
     model.add(LSTM(128, input_shape=net_input.shape[1:], return_sequences=True))  
     model.add(Dropout(0.2))  
     model.add(LSTM(128, return_sequences=True))  
     model.add(Flatten())  
     model.add(Dense(256))  
     model.add(Dropout(0.3))  
     model.add(Dense(num_vocab))  
     model.add(Activation('softmax'))  
     model.compile(loss='categorical_crossentropy', optimizer='adam')

     return model
    
  def train_model(self, model, net_input, net_output, epochs=None, batch_size=None):
    if epochs == None:
      epochs = self.epochs
    
    if batch_size == None:
      batch_size = self.batch_size

    filepath = "music_generation.h5"
    checkpoint = ModelCheckpoint(filepath, monitor="loss", verbose=2, save_best_only=True)

    history = model.fit(net_input, net_output, epochs=epochs, batch_size=batch_size, callbacks=[checkpoint])

    return history
  
  def generate_audio(self, net_input, pitchnames, num_vocab):
    start = np.random.randint(0, len(net_input)-1)

    int_note = dict((number, the_note) for number, the_note in enumerate(pitchnames))

    random_sequence = list(net_input[start])

    model = load_model("music_generation.h5")

    prediction = []

    for i in range(500):
      input_prediction = np.reshape(random_sequence, (1, len(random_sequence), 1))
      input_prediction = input_prediction / float(num_vocab)
      
      input_prediction = np.asarray(input_prediction).astype("float32")

      new_prediction = model.predict(np.asarray(input_prediction), verbose=2)
      


      index = np.argmax(new_prediction)

      result = int_note[index]

      prediction.append(result)

      random_sequence.append(index)

      random_sequence = random_sequence[1:len(random_sequence)]
    
    return prediction

  def get_inputSequences(self, the_notes, pitchnames, num_vocab):
    note_int = dict((the_note, number) for number, the_note in enumerate(pitchnames))

    length_of_seqeunce = 100
    net_input = []
    for i in range(0, len(the_notes) - length_of_seqeunce, 1):
        sequence_in = the_notes[i:i + length_of_seqeunce]
        net_input.append([note_int[note] for note in sequence_in])
    
    net_input = np.reshape(net_input, (len(net_input), 100, 1))
    
    return (net_input)

  def create_midi(self, prediction_output):
    offset = 0
    output_notes = []

    for pattern in prediction_output:

        if ('.' in pattern) or (pattern.isdigit()):
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)

        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    
    print('Saving Output file as midi....')

    midi_stream.write('midi', fp='test_output4.mid')
    print("Done")



    


In [97]:
gen = AudioGenerator()

1/1 - 1s
1/1 - 0s
1/1 - 0s


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1